In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#installing libraries
!pip install streamlit
!pip install openai
!pip install langchain-openai
!pip install langchain
!pip install requests
!pip install googlemaps
!pip install ipywidgets

In [3]:
#importing dependencies
import streamlit as st
import openai
import googlemaps
import requests
import ipywidgets as widgets

from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from openai import OpenAI
from IPython.display import display, clear_output

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAIKEY = user_secrets.get_secret("OPENAIKEY")
googlemaps_api_key = user_secrets.get_secret("googlemaps_api_key")
openweather_api_key = user_secrets.get_secret("openweather_api_key")

In [5]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0, 
                 api_key=OPENAIKEY)

In [6]:
def get_attractions(destination):
    """Fetches a list of tourist attractions near a given destination using the Google Maps API.

    Args:
        destination (str): The name of the destination (e.g., "Paris, France").

    Returns:
        list[str]: A list of names of tourist attractions near the destination.
                   Returns an empty list if no attractions are found.

    Raises:
        ValueError: If the destination cannot be geocoded.
        googlemaps.exceptions.ApiError: If there is an issue with the Google Maps API request.
        KeyError: If the API key is not found in Streamlit secrets.

    Note:
        This function requires the 'googlemaps' and 'streamlit' libraries to be installed
        and a valid Google Maps API key stored in Streamlit secrets under the key
        "googlemaps_api_key". It searches for attractions within a 5km radius
        of the geocoded destination. Requires 'googlemaps' and 'streamlit'
        libraries to be imported.
    """

    gmaps = googlemaps.Client(key=googlemaps_api_key)

    geocode_result = gmaps.geocode(destination)
    if not geocode_result:
        raise ValueError(f"Could not geocode destination: {destination}")
    location = geocode_result[0]['geometry']['location']
    lat_lng = (location['lat'], location['lng'])

    places_result = gmaps.places_nearby(location=lat_lng,radius=5000,type='tourist_attraction')
    return [place['name'] for place in places_result['results']]

In [7]:
def get_weather(destination):
    """Fetches the current weather description for a given destination using OpenWeatherMap API.

    Args:
        destination (str): The city name for which to retrieve the weather.

    Returns:
        str: A string describing the current weather conditions (e.g., 'clear sky').
             Note: This function assumes the API key is stored in Streamlit secrets
             and does not handle potential errors like invalid destinations,
             network issues, or API key problems gracefully. It might raise
             exceptions (e.g., KeyError, requests.exceptions.RequestException)
             if the API call fails or the response format is unexpected. Requires
             'requests' and 'streamlit' libraries to be imported.
    """    
    api_key = openweather_api_key
    url = f"http://api.openweathermap.org/data/2.5/weather?q={destination}&appid={api_key}"
    response = requests.get(url).json()
    return response['weather'][0]['description']

In [8]:
def generate_itinerary(destination, start_date, end_date, budget, interests):
    """Generates a personalized travel itinerary using OpenAI.

    Fetches local attractions and weather forecasts to create a detailed,
    day-by-day plan based on user preferences and constraints.

    Args:
        destination (str): The city or region for the trip.
        start_date (str): The starting date of the trip (e.g., "YYYY-MM-DD").
        end_date (str): The ending date of the trip (e.g., "YYYY-MM-DD").
        budget (float | int): The total budget for the trip in GBP (£).
        interests (list[str]): A list of the traveler's interests (e.g., ["history", "food"]).

    Returns:
        str: A string containing the generated day-by-day itinerary from the OpenAI model.

    Raises:
        openai.APIError: If there is an issue communicating with the OpenAI API.
        # Note: This function also depends on the successful execution of
        # get_attractions() and get_weather(), which might raise their own specific errors
        # (e.g., related to Google Maps API, OpenWeatherMap API, network issues, or missing keys).

    Note:
        Assumes `get_attractions` and `get_weather` functions are defined and
        accessible in the same scope and handle their own API interactions.
    """ 
    tourist_attraction = get_attractions(destination)
    weather_forecast = get_weather(destination)

    client = OpenAI(api_key=OPENAIKEY)
    prompt = f"""
    You are a helpful tour planner.
    Please create a day-by-day itinerary for a trip to {destination} 
    from {start_date} to {end_date}
    within a budget of £{budget} and 
    with focus on the below interests {', '.join(interests)}.
    Please also include places like, {tourist_attraction}, in the itinerary and
    factor the forecasted weather, like {weather_forecast} into your itinerary.
    """
    response = client.chat.completions.create(model="gpt-3.5-turbo-0125",
                                              messages=[
                                                            {
                                                                "role": "developer",
                                                                "content": prompt
                                                            }
                                                        ],
                                              max_tokens=1000,
                                              temperature=0)
    return response.choices[0].message.content

In [9]:
destination = input("Enter your destination:")

Enter your destination: Paris


In [10]:
budget = input("Enter your budget (£):")

Enter your budget (£): 500


In [11]:
# Create the Start-Date DatePicker widget
startdatewidget = widgets.DatePicker(description='Pick a Start Date',
                                     style={'description_width': 'initial'})

# Create the End-Date DatePicker widget
enddatewidget = widgets.DatePicker(description='Pick an End Date',
                                   style={'description_width': 'initial'})

# Define your interests
options = ["Nature", "History", "Food", "Adventure", "Shopping", "Relaxation"]

# Create the Interests SelectMultiple widget
interestswidget = widgets.SelectMultiple(options=options,
                                         description="Choose your interests:",
                                         rows=6,
                                         style={'description_width': 'initial'},
                                         disabled=False)

# Create a button
submit_button = widgets.Button(description="Submit")

# Output area to display selected items
output = widgets.Output()

In [12]:
# On button click
def on_submit_clicked(b):
    with output:
        clear_output()
        start_date = startdatewidget.value
        end_date = enddatewidget.value
        interests = list(interestswidget.value)

        if not start_date or not end_date or not interests:
            print("❗ Please select both trip start and end dates and at least one interest before submitting.")
        else:
            print("Generating your trip plan...")
            itinerary = generate_itinerary(destination, start_date, end_date, budget, interests)
            print("Your AI-generated itinerary:\n ")
            print(itinerary)

submit_button.on_click(on_submit_clicked)

# Display everything
display(startdatewidget, enddatewidget, interestswidget, submit_button, output)

DatePicker(value=None, description='Pick a Start Date', step=1, style=DescriptionStyle(description_width='init…

DatePicker(value=None, description='Pick an End Date', step=1, style=DescriptionStyle(description_width='initi…

SelectMultiple(description='Choose your interests:', options=('Nature', 'History', 'Food', 'Adventure', 'Shopp…

Button(description='Submit', style=ButtonStyle())

Output()